In [1]:
# data_preprocessing.py
from datetime import date, datetime, timedelta
import polars as pl
from deltalake import DeltaTable
import json
from gcsfs import GCSFileSystem
import pyarrow

# custom imports
import sys
sys.path.append('..')
from common.gcp import GCPClient


In [2]:

class DataPreprocessor:
    def __init__(self, gcp_client, bucket: str, table_path: str):
        self.gcp_client = gcp_client
        self.table_path = table_path
        self.bucket = bucket
        self.table_path_full = f'gs://{bucket}/{table_path}'
    
    def get_date_from_lookback(self, lookback_days: int, return_as_str = True):
        target_date = datetime.utcnow() - timedelta(days=lookback_days)
        
        if return_as_str:
            year, month, day = target_date.year, target_date.month, target_date.day
            return f'{year}/{month:02d}/{day:02d}'
        else:
            return target_date

    def read_data(self):

        start_date = self.get_date_from_lookback(lookback_days=1000, return_as_str=True)
        print(f"Reading data from {self.table_path_full} with start date {start_date}")

        gcp_creds_json_str = json.dumps(self.gcp_client.creds_json)
        storage_options = {"service_account_key": gcp_creds_json_str}
        
        dt = DeltaTable(self.table_path_full, storage_options=storage_options)
        # dt = DeltaTable(self.table_path_full, storage_options=storage_options)

        gcp_client = GCPClient()   
        fs = pyarrow.fs.GcsFileSystem(access_token=gcp_client.creds_encoded, credential_token_expiration = datetime.fromisoformat('9999-12-31') )

        df = dt.to_pyarrow_dataset(partitions=[("crt_ts_date", ">=", start_date)])
        # df = pl.scan_pyarrow_dataset(df)

        
        # df = pl.scan_delta(
        #         self.table_path_full,
        #         pyarrow_options={"partitions": [("crt_ts_date", ">=", start_date)]},
        #         storage_options=storage_options,
        # )
        
        return df


    # Primary data processing
    
    def create_target(self, df):
        # Create target variable, time to landing
        landing_times = df.group_by('fa_flight_id').agg(pl.max('actual_in').alias('actual_in_filled'))
        df = df.join(landing_times, on='fa_flight_id')
        df = df.with_columns( ((pl.col('actual_in_filled').dt.timestamp("ms")-pl.col('event_ts').dt.timestamp("ms"))/1000/60/60 ).alias('target') )
        
        df = df.sort(['actual_in_filled','crt_ts'])
        return df

    def remove_incomplete_flights(self, df):
        # Remove flights that haven't landed yet
        return df.filter( pl.col('actual_in_filled').is_not_null() )

    def removed_arrival_events(self, df):
        # Arrival events are not useful for training
        return df.filter(pl.col('event_type') != 'actual_in')

    def process_data(self, df):
        df = self.create_target(df)
        df = self.remove_incomplete_flights(df)
        df = self.removed_arrival_events(df)
        return df
    
    def write_data_to_gcs(self, df, path_out: str):
        print(f"Writing data to {path_out}")
        bucket = self.gcp_client.storage_client.get_bucket(self.bucket) 
        blob = bucket.blob(f'{path_out}.csv')
        blob.upload_from_string(df.collect().write_csv(), 'text/csv')

    def write_data_locally(self, df, path_out: str):
        print(f"Writing data to {path_out}")
        df.collect().write_parquet(path_out)


In [3]:
gcp_client = GCPClient()

project_id = 'aia-ds-accelerator-flight-1'
bucket = 'datalake-flight-dev-1'
table_path_in = 'flightsummary-delta-processed-stream'
table_path_out = 'flightsummary-delta-processed-training'

data_preprocessor = DataPreprocessor(gcp_client, bucket=bucket, table_path=table_path_in)

In [4]:
df = data_preprocessor.read_data()
df = pl.scan_pyarrow_dataset(df)
df = data_preprocessor.process_data(df)
data_preprocessor.write_data_to_gcs(df, table_path_out)

Reading data from gs://datalake-flight-dev-1/flightsummary-delta-processed-stream with start date 2021/02/10


<LazyFrame [76 cols, {"ident": Utf8 … "event_ts": Datetime(time_unit='us', time_zone=None)}] at 0x10572F0A0>

In [ ]:
# # Step 3: Write data
data_preprocessor.write_data_to_gcs(df_processed, path_out=table_path_out)